# Install Dependencies

In [ ]:
!pip install opencv-python numpy diffusers gradio

# Import Library

In [ ]:
import cv2
import numpy as np
from diffusers import AutoPipelineForInpainting, AutoencoderKL
from diffusers.utils import load_image
import torch
from PIL import Image
import numpy as np

# Image Processing

In [ ]:
class ImageMosaicExtender:
    def __init__(self, block_size=20, transition_width=40):
        """
        Initialize the mosaic extender with given parameters.

        Args:
            block_size (int): Size of mosaic blocks
            transition_width (int): Width of the smooth transition
        """
        self.block_size = block_size
        self.transition_width = transition_width

    def create_sigmoid_mask(self, size, direction='right'):
      """
      Create a sigmoid-based transition mask with black background and white transition.

      Args:
          size (tuple): Size of the mask (height, width)
          direction (str): Direction of transition ('left', 'right', 'top', 'bottom')

      Returns:
          np.ndarray: A 2D mask with black background and white transition.
      """
      h, w = size
      mask = np.zeros((h, w), dtype=np.float32)

      x = np.linspace(-6, 6, self.transition_width)  # Sigmoid range
      sigmoid = 1 / (1 + np.exp(-x))  # Sigmoid function

      if direction == 'left':
          for i in range(min(w, self.transition_width)):
              mask[:, i] = sigmoid[self.transition_width - i - 1]  # Gradually transition
      elif direction == 'right':
          for i in range(min(w, self.transition_width)):
              mask[:, -(i + 1)] = sigmoid[self.transition_width - i - 1]
      elif direction == 'top':
          for i in range(min(h, self.transition_width)):
              mask[i, :] = sigmoid[self.transition_width - i - 1]
      elif direction == 'bottom':
          for i in range(min(h, self.transition_width)):
              mask[-(i + 1), :] = sigmoid[self.transition_width - i - 1]

      return mask


    def create_mosaic_pattern(self, img_section):
        """
        Create mosaic pattern from an image section.
        """
        h, w = img_section.shape[:2]
        result = np.zeros_like(img_section, dtype=np.float32)

        for y in range(0, h, self.block_size):
            for x in range(0, w, self.block_size):
                y_end = min(y + self.block_size, h)
                x_end = min(x + self.block_size, w)
                block = img_section[y:y_end, x:x_end]
                color = np.mean(block, axis=(0, 1))
                result[y:y_end, x:x_end] = color

        return result

    def extend_image(self, image_path, extensions):
        """
        Extend image with mosaic effect in multiple directions.

        Args:
            image_path (str): Path to input image
            extensions (dict): Dictionary with extension sizes for each direction
                             e.g., {'left': 100, 'right': 200, 'top': 50, 'bottom': 50}
        """
        # Read and convert image to float32 for processing
        # img = cv2.imread(image_path).astype(np.float32)
        img = image_path
        if img is None:
            raise ValueError("Could not read the image")

        h, w = img.shape[:2]

        # Calculate new dimensions
        new_h = h + extensions.get('top', 0) + extensions.get('bottom', 0)
        new_w = w + extensions.get('left', 0) + extensions.get('right', 0)

        # Create extended image
        extended = np.zeros((new_h, new_w, 3), dtype=np.float32)

        # Position of original image in extended canvas
        y_offset = extensions.get('top', 0)
        x_offset = extensions.get('left', 0)

        # Place original image
        extended[y_offset:y_offset+h, x_offset:x_offset+w] = img



        # Process each direction
        masks = {}
        for direction, size in extensions.items():
            if size == 0:
                continue

            if direction == 'left':
                section = extended[y_offset:y_offset+h, :x_offset]
                base_color = img[:, :1]
            elif direction == 'right':
                section = extended[y_offset:y_offset+h, x_offset+w:]
                base_color = img[:, -1:]
            elif direction == 'top':
                section = extended[:y_offset, x_offset:x_offset+w]
                base_color = img[:1, :]
            else:  # bottom
                section = extended[y_offset+h:, x_offset:x_offset+w]
                base_color = img[-1:, :]

            # Create mosaic pattern
            if direction in ['left', 'right']:
                base = np.tile(base_color, (1, size, 1))
            else:
                base = np.tile(base_color, (size, 1, 1))

            mosaic = self.create_mosaic_pattern(base)

            # Create and store transition mask
            mask = self.create_sigmoid_mask(section.shape[:2], direction)
            masks[direction] = np.expand_dims(mask, axis=2)

            # Apply transition
            if direction == 'left':
                extended[y_offset:y_offset+h, :x_offset] = \
                    mosaic * masks[direction] + base * (1 - masks[direction])
            elif direction == 'right':
                extended[y_offset:y_offset+h, x_offset+w:] = \
                    mosaic * masks[direction] + base * (1 - masks[direction])
            elif direction == 'top':
                extended[:y_offset, x_offset:x_offset+w] = \
                    mosaic * masks[direction] + base * (1 - masks[direction])
            else:  # bottom
                extended[y_offset+h:, x_offset:x_offset+w] = \
                    mosaic * masks[direction] + base * (1 - masks[direction])

        mask_image = np.ones_like(extended, dtype=np.float32)
        mask_image[y_offset:y_offset+h, x_offset:x_offset+w] = np.zeros_like(img)
        # cv2.imwrite('mask_image.jpg', (mask_image * 255).astype(np.uint8))
        return extended.astype(np.uint8), (mask_image * 255).astype(np.uint8)

# Stabel Diffusion

In [ ]:
def convert_to_pil(input_image):
    # Convert the NumPy array (HWC, BGR format) to RGB format for PIL
    if input_image.ndim == 3:  # Color image
        pil_image = Image.fromarray(input_image.astype("uint8"), "RGB")
    else:  # Grayscale image
        pil_image = Image.fromarray(input_image.astype("uint8"), "L")
    return pil_image

def inpaint_image(input_image_path, mask_image_path, prompt):
  """
  Inpaint an image using a pre-trained diffusion model.

  Args:
    input_image_path (str): Path to the input image.
    mask_image_path (str): Path to the mask image.

  Returns:
    PIL.Image: The inpainted image.
  """

  # Convert the NumPy array from Gradio to PIL Image
  if isinstance(input_image_path, np.ndarray):
        input_image = convert_to_pil(input_image_path)
  # Covert mask image
  if isinstance(mask_image_path, np.ndarray):
        mask_image = convert_to_pil(mask_image_path)
  output_size=(1024, 1024)
  guidance_scale=8.0
  num_inference_steps=25 
  strength=0.99
  seed=42
  vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
  pipeline = AutoPipelineForInpainting.from_pretrained("diffusers/stable-diffusion-xl-1.0-inpainting-0.1", torch_dtype=torch.float16, variant="fp16", vae=vae).to("cuda")
  pipeline.load_ip_adapter("h94/IP-Adapter", subfolder="sdxl_models", weight_name="ip-adapter_sdxl.bin", low_cpu_mem_usage=True)

  input_image = load_image(input_image)
  mask_image = load_image(mask_image)
  ip_image = input_image.convert("RGB")

  w,h = input_image.size
  resize_input_image = input_image.resize(output_size)
  resize_mask_image = mask_image.resize(output_size)

  generator = torch.Generator(device="cuda").manual_seed(seed)

  inpainted_image = pipeline(
    prompt=prompt,
    image=resize_input_image,
    mask_image=resize_mask_image,
    ip_adapter_image=ip_image,
    guidance_scale=guidance_scale,
    num_inference_steps=num_inference_steps,
    strength=strength,
    generator=generator,
  ).images[0]

  return inpainted_image.resize((w,h))

# Running Gradio

In [ ]:
import gradio as gr
import numpy as np

# Define a function that takes an image and several checkboxes and a slider as input
# and returns the image twice as output. The function currently does not use the checkboxes or slider.
def image_processing(input_image, direction, pixel_extended):
    """
    Process the uploaded image using the mosaic extender and return the processed images.
    Args:
        input_image: Uploaded image (from Gradio)
        direction: Direction of extension ('left', 'right', 'top', 'bottom')
        pixel_extended: Number of pixels to extend in the specified direction
        block_size: Size of mosaic blocks
        transition_width: Width of the smooth transition
    Returns:
        tuple: Masked image and mosaic-extended image
    """
    block_size=20
    transition_width=40

    # Check if the input is a NumPy array (Gradio typically passes NumPy arrays for images)
    if isinstance(input_image, np.ndarray):
        input_image = input_image.astype(np.float32)
    else:
        raise ValueError("Expected input_image to be a NumPy array.")

    # Set up extensions
    extensions = {'left': 0, 'right': 0, 'top': 0, 'bottom': 0}
    extensions[direction] = pixel_extended

    # Initialize the mosaic extender
    mosaic_extender = ImageMosaicExtender(block_size=block_size, transition_width=transition_width)

    # Extend the image and get the mask
    extended_image, mask_image = mosaic_extender.extend_image(input_image, extensions)

    return mask_image, extended_image

# Define a function that takes two images as input and returns a new image
def process_images(mask_image, mosaic_image, prompt):
    extended_image = inpaint_image(input_image_path=mosaic_image, mask_image_path=mask_image, prompt=prompt)
    return extended_image

# Create a Gradio Interface
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            input_image = gr.Image(label="Upload Image")
            direction = gr.Radio(choices=['left', 'right', 'top', 'bottom'], label="Direction")
            pixel_extended = gr.Slider(0, 700, label="Pixel Extension", value=50, step=10, info="Choose between 0 and 200")
            prompt = gr.Textbox(label='Prompt for Image Extender', lines=3, value="fill the background color very consious and perfectly")
        with gr.Column():
            mosaic_image = gr.Image(label="Mosaic Image")
            mask_image = gr.Image(label="Masked Image")
        final_output_image = gr.Image(label="Final Processed Image")

    # Function to check if the image is valid before triggering the timer
    def start_timer_if_image_uploaded(image):
        # If the image is uploaded, activate the timer
        return gr.Timer(active=True) if image is not None else gr.Timer(active=False)
    # Create a timer that triggers the greet function 2 seconds after the slider value changes
    timer = gr.Timer(2, active=False)
    # Ensure the timer only activates when the input image is valid
    input_image.change(start_timer_if_image_uploaded, [input_image], timer)
    # Process the image after timer delay
    timer.tick(image_processing, [input_image, direction, pixel_extended], [mask_image, mosaic_image])

    # Create a button to process the images
    process_button = gr.Button("Process Images")
    process_button.click(process_images, [mask_image, mosaic_image, prompt], final_output_image)

demo.launch(show_error=True, debug=True)